## Exemple d'utilisation de réseaux RNN 

https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/Basics/pytorch_rnn_gru_lstm.py

In [4]:
import torch 
import torchvision 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
train_dataset = datasets.MNIST(root='dataset/',train=True, transform=transforms.ToTensor(),download=True)
test_dataset = datasets.MNIST(root='dataset/',train=False, transform=transforms.ToTensor(),download=True)

100%|██████████| 9912422/9912422 [00:11<00:00, 894138.20it/s] 


Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 1192010.61it/s]


Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw



100%|██████████| 1648877/1648877 [00:09<00:00, 166199.28it/s] 


Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 540011.59it/s]


Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw



In [13]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = True)

## DNN

In [ ]:
input_size = train_dataset.data.shape[1]*train_dataset.data.shape[2] #784
batch_size = 64
num_classes = 10
learning_rate = 10e-3
num_epochs = 1

In [27]:
class NN(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc1 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [28]:
#Initialisation 
model = NN(input_size = input_size, num_classes=num_classes).to(device)

In [30]:
#Optimisation
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

## RNN

In [44]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 10e-3
num_epochs = 2
batch_size = 64

In [48]:
class RNN(nn.Module): 
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        #batch_sizextime_sequencextime_features
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.rnn(x,h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [49]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [52]:
#Entrainenemt
for epoch in range(num_epochs): 
    for batch_idx, (data, targets) in enumerate(train_loader): 
        #get data to cuda if possible 
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        
        #forward
        scores = model(data)
        loss  = criterion(scores, targets)
        
        #backward 
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

In [53]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 7.253333
Accuracy on test set: 7.25


In [54]:
# Recurrent neural network with GRU (many-to-one)
class RNN_GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out


# Recurrent neural network with LSTM (many-to-one)
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out
